In [460]:
from pymongo import MongoClient
import pprint
import indicoio as ico 
ico.config.api_key = "07ce0e1a82169f11530188d15042c10a"
import pandas as pd
from pandas import DataFrame
from pandas.io.json import json_normalize
import textcleaner as tc
import nltk
nltk.download('wordnet')
import collections
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import seaborn as sns
from __future__ import print_function
import gensim
from gensim import corpora, models


[nltk_data] Error loading wordnet: <urlopen error [WinError 10060]
[nltk_data]     Попытка установить соединение была безуспешной, т.к.
[nltk_data]     от другого компьютера за требуемое время не получен
[nltk_data]     нужный отклик, или было разорвано уже установленное
[nltk_data]     соединение из-за неверного отклика уже подключенного
[nltk_data]     компьютера>


In [462]:
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [428]:
client = MongoClient('mongodb://localhost:27017/myDB')
db = client['mydb']
wall_posts = db['wall_posts']    

In [429]:
df = json_normalize(list(wall_posts.find({}))).drop_duplicates(subset=['text'], keep='first', inplace=False)
df['Sentiment Rate'] = df.text.apply(lambda x: (ico.sentiment(x)) if x!= "" else None )

In [430]:
stop_ru, stop_en = set(stopwords.words('russian')), set(stopwords.words('english'))
def del_stops(texts):
    for text  in texts:
        for word in text:
            if (word in stop_ru) or (word in stop_en):
                text.remove(word)

In [431]:
#drop rows with empty text
df = df.drop(df[df.text == ''].index, inplace = False)
df['text_clean'] = df.text.apply(lambda x: tc.main_cleaner(x) )
df['text_clean'] = df.text_clean.apply(lambda x: " ".join(x).split())
del_stops(df['text_clean'])
df = df.drop(columns = ['owner_id','marked_as_ads','post_type','_id','owner_id','id', 'from_id', 'post_source.type'])

In [433]:
df["Sentiment"] = df['Sentiment Rate'].apply(lambda x: "Positive" if x>0.5 else "Negative" )

In [434]:
final_df = final_df.append(df)
final_df = final_df.drop_duplicates(subset=['text'], keep='first', inplace=False)

C:\Users\kirill\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [464]:
final_df.describe()
final_df['Most Common Words'] = final_df['text_clean'].apply(lambda x: collections.Counter(x).most_common(5)) 
final_df.head()
final_df.to_csv('out.csv')

In [450]:
all_words = []
for text in final_df['text_clean']:
    all_words.append(text)
all_words[0]       

['продажа',
 'обмен',
 'лич',
 'встреча',
 'екатеринбург',
 'метро',
 'чкаловская',
 'kappa',
 'olimp',
 'размер',
 'м',
 'состояние',
 'цена',
 'лс',
 'возможен',
 'обмен',
 'доп',
 'информацией',
 'лс']

In [455]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(all_words)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in all_words]

In [456]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, passes=20)
import pprint
pprint.pprint(ldamodel.top_topics(corpus,topn=5))

[([(0.019231101, 'музей'),
   (0.014494469, 'театр'),
   (0.011578962, 'ул'),
   (0.011068594, 'д'),
   (0.010544493, 'пр')],
  -0.5496317832966977),
 ([(0.007905437, 'адрес'),
   (0.0055247913, 'руб'),
   (0.0050732093, 'область'),
   (0.004873262, 'месяц'),
   (0.0045787427, 'комиссии')],
  -1.2786322053078716),
 ([(0.005573561, 'чкаловский'),
   (0.004497452, 'жители'),
   (0.0037568253, 'космическому'),
   (0.0037567404, 'проспекту'),
   (0.003756615, 'план')],
  -1.3956670698534022),
 ([(0.014090367, 'р'),
   (0.0052234256, 'метро'),
   (0.004321379, 'м'),
   (0.0043008267, 'х'),
   (0.0042024455, 'в')],
  -1.8702941472288608),
 ([(0.0067710835, 'д'),
   (0.0067707617, 'адрес'),
   (0.006024676, 'м'),
   (0.00602368, 'кино'),
   (0.0060234647, 'показы')],
  -2.171381977167364),
 ([(0.025386289, 'м'),
   (0.010319721, 'чкаловский'),
   (0.008438007, 'этаж'),
   (0.006333847, 'ул'),
   (0.006054474, 'чкаловская')],
  -3.818296918212244),
 ([(0.01808139, 'м'),
   (0.00865577, 'двор')

In [463]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

C:\Users\kirill\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.274524 -0.047567       1        1  17.342686
8      0.016393 -0.160023       2        1  15.433447
2     -0.055075  0.221391       3        1  12.446174
9      0.044266  0.002299       4        1   6.602387
1      0.048899  0.069876       5        1   6.256295
0      0.040296 -0.013775       6        1   5.817868
3      0.010588  0.007012       7        1   5.129072
7      0.033832 -0.008977       8        1   4.860650
14     0.002463 -0.001616       9        1   4.306991
10     0.029108 -0.004876      10        1   4.271300
6      0.020634 -0.011565      11        1   4.043740
5      0.049777 -0.008598      12        1   4.002323
12    -0.006802 -0.027269      13        1   4.001606
11     0.014640 -0.006158      14        1   3.940215
4      0.025506 -0.010154      15        1   1.545241, topic_info=     Category        Freq                      Term       Total  loglift  \
term                                                                       
9     Default  105.000000                         м  105.000000  30.0000   
342   Default   69.000000                     музей   69.000000  29.0000   
1652  Default   46.000000                     театр   46.000000  28.0000   
6030  Default   28.000000                       buy   28.000000  27.0000   
58    Default   44.000000                        пр   44.000000  26.0000   
49    Default   51.000000                         д   51.000000  25.0000   
47    Default   29.000000                     адрес   29.000000  24.0000   
1013  Default   20.000000                         р   20.000000  23.0000   
506   Default   21.000000                      двор   21.000000  22.0000   
271   Default   73.000000                        ул   73.000000  21.0000   
21    Default   34.000000                чкаловский   34.000000  20.0000   
16    Default   47.000000                чкаловская   47.000000  19.0000   
216   Default   25.000000                  проспект   25.000000  18.0000   
2768  Default   10.000000                         ️   10.000000  17.0000   
1071  Default   26.000000                       дом   26.000000  16.0000   
821   Default   10.000000                      этаж   10.000000  15.0000   
15    Default   16.000000                      цена   16.000000  14.0000   
479   Default   13.000000                       наш   13.000000  13.0000   
292   Default   10.000000                     шоссе   10.000000  12.0000   
272   Default   19.000000                     улица   19.000000  11.0000   
518   Default   11.000000                  квартира   11.000000  10.0000   
44    Default   18.000000                        ст   18.000000   9.0000   
10    Default   24.000000                     метро   24.000000   8.0000   
1751  Default    9.000000                   область    9.000000   7.0000   
296   Default    6.000000                 эвакуатор    6.000000   6.0000   
1591  Default   12.000000                       руб   12.000000   5.0000   
7938  Default   11.000000                    дворик   11.000000   4.0000   
101   Default   28.000000                      года   28.000000   3.0000   
197   Default   23.000000                      парк   23.000000   2.0000   
984   Default   35.000000                     место   35.000000   1.0000   
...       ...         ...                       ...         ...      ...   
6982  Topic15    0.342092                   дуброва    0.938875   3.1604   
6983  Topic15    0.342092  калинковичирайагросервис    0.938875   3.1604   
6986  Topic15    0.342092                нанимателе    0.938875   3.1604   
6979  Topic15    0.342092                 бухгалтер    0.938875   3.1604   
6978  Topic15    0.342092                        cc    0.938875   3.1604   
6996  Topic15    0.342092                  характер    0.938875   3.1604   
35    Topic15    0.983432               посредников    2.711698   3.1557   